# Bernstein-Vazirani and Simon algorithms

In [ ]:
# To use galois in the Simon algorithm, we need to downgrade NumPy to 2.0
import sys
!{sys.executable} -m pip install --upgrade numpy==2.0
!{sys.executable} -m pip install numba --upgrade
!{sys.executable} -m pip install galois

In [3]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import UnitaryGate
from qiskit_aer import *
import numpy as np
import galois

from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Choosing local simulation to run circuits
service = QiskitRuntimeService(channel="local")

## Bernstein-Vazirani

You can use the circuit defined for Deutsch-Josza algorithm. Two adjustments must be made:
1) The oracle must be adapted to depend on a bitstring c (the function f in Deutsch oracle is f(x)=c.x (that is, inner product mod 2, equivalent to numpy bitwise_xor)
2) The output is the whole bitstring measured by the circuit (instead of just checking if the measure is 0000...0).

## Simon

The following function defines, given a bitstring mask s, a random $f:\{ 0,1 \}^n\to \{0,1\}^n$ such that $f(x)=f(y)\iff x=y\oplus s$.

In [4]:
def Simon_f(s:str):
    n = len(s)
    per = np.random.permutation(n)
    def f(x):
        out1 = min (x, x ^ int(s,2))
        out1 = f'{out1:0{n}b}'
        out2 = ""
        for i in range(n):
            out2 = out2 + out1[per[i]]
        return(int(out2,2))
    return f   

In [6]:
g = Simon_f("1001")
print(g(3),g(10),g(11))

5 5 4


Now you must construct an XOR oracle associated to a function $f:\{ 0,1 \}^n\to \{0,1\}^n$ of $n$ bits.

Using the explicit expression of the oracle in terms of output of any input, $$U_f |x,y\rangle = |x,y\oplus f(x)\rangle,$$ then
$$U_f = \sum_{x\in \{0,1\}^n} \sum _{y\in \{0,1\}^n} |x,y\oplus f(x) \rangle\langle x,y|.$$

(Remember again, qiskit is little-endian)

Then, you have to put the oracle inside of the full Simon's algorithm circuit and run it with $n+10$ shots (for error probability smaller than $10^{-3}$).

You can use something as

```result = AerSimulator().run(quantum_circuit_name, shots=n+10, memory=True).result()```

and access the measures with 

```result.get_memory()```

Finally, we need a procedure that classically solves the linear system (mod 2) posed by the $n+10$ measures. That is in turn equivalent to compute a nonzero vector of the nullspace of the matrix given by all the measures. You can use the ```galois``` library to use finite fields in ```numpy```

Somthing along the lines of

``` matrix = np.array([list(bitstring) for bitstring in measurements]).astype(int) ```

``` null_space = galois.GF(2)(matrix).null_space() ```

``` display(null_space) ```
